In [4]:
import pandas as pd

movies_df = pd.read_csv(r"C:\Users\JOHN\Downloads\IMDb Movies India\IMDb Movies India.csv", encoding='latin-1')

print(movies_df.head())
#We notice that Year, and Duration need conversion to int

                                 Name    Year Duration            Genre  \
0                                         NaN      NaN            Drama   
1  #Gadhvi (He thought he was Gandhi)  (2019)  109 min            Drama   
2                         #Homecoming  (2021)   90 min   Drama, Musical   
3                             #Yaaram  (2019)  110 min  Comedy, Romance   
4                   ...And Once Again  (2010)  105 min            Drama   

   Rating Votes            Director       Actor 1             Actor 2  \
0     NaN   NaN       J.S. Randhawa      Manmauji              Birbal   
1     7.0     8       Gaurav Bakshi  Rasika Dugal      Vivek Ghamande   
2     NaN   NaN  Soumyajit Majumdar  Sayani Gupta   Plabita Borthakur   
3     4.4    35          Ovais Khan       Prateik          Ishita Raj   
4     NaN   NaN        Amol Palekar  Rajat Kapoor  Rituparna Sengupta   

           Actor 3  
0  Rajendra Bhatia  
1    Arvind Jangid  
2       Roy Angana  
3  Siddhant Kapoor  
4    

In [14]:
print(movies_df['Votes'].dtype)
#Votes need to be converted to numeric as well

object


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

class SafeLabelEncoder(LabelEncoder):
    def fit(self, y):
        super().fit(y)
        self.classes_ = np.append(self.classes_, '<unknown>')
        return self
    
    def transform(self, y):
        new_y = []
        for label in y:
            if label in self.classes_:
                new_y.append(label)
            else:
                new_y.append('<unknown>')
        return super().transform(new_y)
    
    def fit_transform(self, y):
        return self.fit(y).transform(y)


df = pd.read_csv(r"C:\Users\JOHN\Downloads\IMDb Movies India\IMDb Movies India.csv", encoding='latin-1')

year = input("Enter Movie Release year")

duration = input("Enter Movie length")
genre = input("Enter Movie's genre")
director = input("Enter Movie's Director")

actor_One = input("Enter Movie's lead actor")
actor_Two = input("Enter Movie's secodary")
actor_Three = input("Enter Movie's tertiary actor")
votes = input("How many voted for this movie : ")

# Removing rows with missing rating values
df = df.dropna(subset=['Rating'])

# Convert Year values to numeric
df['Year'] = df['Year'].str.extract('(\d{4})').astype(float)

# Convert Duration values to numeric
df['Duration'] = df['Duration'].str.replace(' min', '').astype(float)

# Convert Votes values to numeric 
df['Votes'] = df['Votes'].str.replace(',', '').astype(float)

# Filling missing numerical values with the median
numerical_columns = ['Year', 'Duration', 'Votes']
for col in numerical_columns:
    df[col] = df[col].fillna(df[col].median())

# Filling missing categorical values with the mode (most repeated value)
categorical_columns = ['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3']
for col in categorical_columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# Converting categorical variables to numerical
label_encoders = {}
for col in categorical_columns:
    le = SafeLabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Splitting the data into training and testing sets
X = df.drop(['Rating', 'Name'], axis=1)
y = df['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Linear Regression model
#lr_model = LinearRegression()
#lr_model.fit(X_train, y_train)

# Hyperparameter tuning for random forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

rf_grid_search = GridSearchCV(estimator=RandomForestRegressor(), param_grid=rf_param_grid, cv=3, n_jobs=-1, verbose=2)
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_

# Training gradient boosting model
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

# Evaluating the models
#lr_predictions = lr_model.predict(X_test)
rf_predictions = rf_best_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)

#lr_mae = mean_absolute_error(y_test, lr_predictions)
#lr_mse = mean_squared_error(y_test, lr_predictions)

rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)

gb_mae = mean_absolute_error(y_test, gb_predictions)
gb_mse = mean_squared_error(y_test, gb_predictions)

# Printing results
print("Model Evaluation Results:\n")
#print(f"Linear Regression Model:")
#print(f"  Mean Absolute Error (MAE): {lr_mae:.2f}")
#print(f"  Mean Squared Error (MSE): {lr_mse:.2f}")

print("\nRandom Forest Model:")
print(f"  Mean Absolute Error (MAE): {rf_mae:.2f}")
print(f"  Mean Squared Error (MSE): {rf_mse:.2f}")

print("\nGradient Boosting Model:")

print(f"  Mean Absolute Error (MAE): {gb_mae:.2f}")
print(f"  Mean Squared Error (MSE): {gb_mse:.2f}")


def predict_rating(year, duration, genre, director, actor1, actor2, actor3, votes):
    # Prepare input data
    input_data = pd.DataFrame({
        'Year': [year],
        'Duration': [duration],
        'Genre': [label_encoders['Genre'].transform([genre])[0]],
        'Director': [label_encoders['Director'].transform([director])[0]],
        'Actor 1': [label_encoders['Actor 1'].transform([actor1])[0]],
        'Actor 2': [label_encoders['Actor 2'].transform([actor2])[0]],
        'Actor 3': [label_encoders['Actor 3'].transform([actor3])[0]],
        'Votes': [votes]
    })

    # Ensures thecolumns are in the same order as training
    input_data = input_data[X_train.columns]

    
    #lr_pred = lr_model.predict(input_data)[0]
    rf_pred = rf_best_model.predict(input_data)[0]
    gb_pred = gb_model.predict(input_data)[0]

    avg_pred = (rf_pred + gb_pred) / 2
    
    # Printing predictions
    print("\nPredicted Ratings:")
    #print(f"  Linear Regression: {lr_pred:.2f}")
    print(f"  Random Forest: {rf_pred:.2f}")
    print(f"  Gradient Boosting: {gb_pred:.2f}")
    print(f"  Average Prediction: {avg_pred:.2f}")

# Example usage of the prediction function

predict_rating(year,duration,genre,director,actor_One,actor_Two,actor_Three,votes)


Enter Movie Release year 1951
Enter Movie length 143
Enter Movie's genre Crime, Drama, Musical
Enter Movie's Director Guru Dutt
Enter Movie's lead actor Dev Anand
Enter Movie's secodary Geeta Bali
Enter Movie's tertiary actor Kalpana Kartik
How many voted for this movie :  343


Fitting 3 folds for each of 27 candidates, totalling 81 fits
Model Evaluation Results:


Random Forest Model:
  Mean Absolute Error (MAE): 0.83
  Mean Squared Error (MSE): 1.22

Gradient Boosting Model:
  Mean Absolute Error (MAE): 0.83
  Mean Squared Error (MSE): 1.18

Predicted Ratings:
  Random Forest: 7.20
  Gradient Boosting: 7.31
  Average Prediction: 7.26


In [20]:
#After googling famous indian directors I decided to use Guru Dutts movies for testing
print(movies_df[movies_df['Director'] == 'Guru Dutt'])

                  Name    Year Duration                       Genre  Rating  \
459           Aar-Paar  (1954)  146 min  Musical, Romance, Thriller     6.6   
1520              Baaz  (1953)  140 min  Action, Adventure, Musical     7.3   
1525             Baazi  (1951)  143 min       Crime, Drama, Musical     7.3   
6267              Jaal  (1952)  165 min             Crime, Thriller     7.2   
6960   Kaagaz Ke Phool  (1959)  148 min                     Romance     8.0   
9536    Mr. & Mrs. '55  (1955)  157 min    Comedy, Musical, Romance     7.2   
10849           Picnic  (1964)      NaN      Drama, Family, Romance     NaN   
11208           Pyaasa  (1957)  146 min     Drama, Musical, Romance     8.4   
12218          Sailaab  (1956)      NaN               Drama, Family     7.9   

       Votes   Director         Actor 1         Actor 2         Actor 3  
459      250  Guru Dutt          Shyama       Guru Dutt   Jagdish Sethi  
1520      93  Guru Dutt       Guru Dutt      Geeta Bali      